# 🚀 High-N Investigation LITE (Skip 1-Form)

**Fast version** - focuses on 0-form spectrum and topology.

Skips the slow 1-form Laplacian computation.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix, lil_matrix, diags, eye
from scipy.sparse.linalg import eigsh
from scipy.sparse.csgraph import connected_components
import gc, time, json
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-whitegrid')
print(f"✓ Setup - {datetime.now().strftime('%H:%M')}")

In [ ]:
# =============================================================================
# CORE FUNCTIONS (compact)
# =============================================================================

def sample_TCS(n, seed):
    rng = np.random.default_rng(seed)
    theta = rng.uniform(0, 2*np.pi, n).astype(np.float32)
    q1 = rng.standard_normal((n, 4)).astype(np.float32)
    q1 /= np.linalg.norm(q1, axis=1, keepdims=True)
    q2 = rng.standard_normal((n, 4)).astype(np.float32)
    q2 /= np.linalg.norm(q2, axis=1, keepdims=True)
    return theta, q1, q2

def compute_knn_chunked(theta, q1, q2, ratio, k=50, chunk_size=2500):
    """Compute k-NN in chunks."""
    n = len(theta)
    alpha = (65/32) / (ratio**3)
    
    knn_idx = np.zeros((n, k), dtype=np.int32)
    knn_dist = np.zeros((n, k), dtype=np.float32)
    
    n_chunks = (n + chunk_size - 1) // chunk_size
    
    for c in range(n_chunks):
        i0, i1 = c * chunk_size, min((c+1) * chunk_size, n)
        
        # S¹ distance
        diff_s1 = np.abs(theta[i0:i1, None] - theta[None, :])
        d_s1 = np.minimum(diff_s1, 2*np.pi - diff_s1)
        
        # S³ distances
        d_s3_1 = 2 * np.arccos(np.clip(np.abs(q1[i0:i1] @ q1.T), 0, 1))
        d_s3_2 = 2 * np.arccos(np.clip(np.abs(q2[i0:i1] @ q2.T), 0, 1))
        
        # TCS metric
        D = np.sqrt(alpha * d_s1**2 + d_s3_1**2 + (ratio**2) * d_s3_2**2)
        
        # Self = inf
        for j in range(i1 - i0):
            D[j, i0 + j] = np.inf
        
        # k-NN
        for j in range(i1 - i0):
            idx = np.argpartition(D[j], k)[:k]
            knn_idx[i0 + j] = idx
            knn_dist[i0 + j] = D[j, idx]
        
        del d_s1, d_s3_1, d_s3_2, D
        gc.collect()
    
    return knn_idx, knn_dist

def build_laplacian(knn_idx, knn_dist):
    n, k = knn_idx.shape
    sigma = float(np.median(knn_dist))
    
    W = lil_matrix((n, n), dtype=np.float32)
    for i in range(n):
        for j_idx in range(k):
            j = knn_idx[i, j_idx]
            w = np.exp(-knn_dist[i, j_idx]**2 / (2*sigma**2))
            W[i, j] = W[j, i] = w
    
    W = W.tocsr()
    d = np.array(W.sum(1)).flatten()
    d_inv_sqrt = 1.0 / np.sqrt(np.maximum(d, 1e-10))
    L = eye(n) - diags(d_inv_sqrt) @ W @ diags(d_inv_sqrt)
    
    return L.tocsr(), W, sigma

print("✓ Core functions")

In [ ]:
# =============================================================================
# ANALYSIS FUNCTIONS
# =============================================================================

def participation_ratio(v):
    v = v.flatten() / np.linalg.norm(v.flatten())
    return float(1.0 / (len(v) * np.sum(v**4)))

def mode_fourier_s1(v, theta):
    """Fourier components on S¹."""
    p = np.abs(v.flatten())**2
    p /= np.sum(p)
    f1 = np.sqrt(np.sum(p*np.cos(theta))**2 + np.sum(p*np.sin(theta))**2)
    f2 = np.sqrt(np.sum(p*np.cos(2*theta))**2 + np.sum(p*np.sin(2*theta))**2)
    f3 = np.sqrt(np.sum(p*np.cos(3*theta))**2 + np.sum(p*np.sin(3*theta))**2)
    return {"F1": float(f1), "F2": float(f2), "F3": float(f3)}

def graph_topology(W, n):
    n_edges = W.nnz // 2
    n_comp, _ = connected_components(W, directed=False)
    beta_1 = n_edges - n + n_comp
    return {"edges": n_edges, "components": n_comp, "beta_1": beta_1}

print("✓ Analysis functions")

In [ ]:
# =============================================================================
# FAST PIPELINE
# =============================================================================

def analyze_fast(N, ratio, seed=42, H_star=99):
    print(f"\n{'='*50}\n  N={N}, ratio={ratio:.2f}\n{'='*50}")
    t0 = time.time()
    
    # Sample
    theta, q1, q2 = sample_TCS(N, seed)
    
    # k-NN
    print("  k-NN...", end=" ", flush=True)
    knn_idx, knn_dist = compute_knn_chunked(theta, q1, q2, ratio)
    print("done")
    
    # Laplacian
    print("  Laplacian...", end=" ", flush=True)
    L, W, sigma = build_laplacian(knn_idx, knn_dist)
    del knn_idx, knn_dist; gc.collect()
    print(f"done (σ={sigma:.4f})")
    
    # Spectrum
    print("  Spectrum...", end=" ", flush=True)
    eigs, vecs = eigsh(L, k=15, which='SM', tol=1e-8)
    idx = np.argsort(eigs)
    eigs, vecs = eigs[idx], vecs[:, idx]
    
    mu1 = float(eigs[1]) if eigs[1] > 1e-8 else float(eigs[2])
    v1 = vecs[:, 1] if eigs[1] > 1e-8 else vecs[:, 2]
    lambda1_hat = mu1 / (sigma**2)
    product = lambda1_hat * H_star
    print(f"done (λ̂₁×H*={product:.4f})")
    
    # Mode analysis
    pr = participation_ratio(v1)
    fourier = mode_fourier_s1(v1, theta)
    
    # Topology
    topo = graph_topology(W, N)
    
    print(f"  PR={pr:.3f}, F1={fourier['F1']:.3f}, F2={fourier['F2']:.3f}")
    print(f"  β₁={topo['beta_1']}, edges={topo['edges']}")
    print(f"  Time: {time.time()-t0:.1f}s")
    
    # Cleanup
    result = {
        "N": N, "ratio": ratio, "sigma": sigma,
        "mu1": mu1, "lambda1_hat": lambda1_hat, "product": product,
        "PR": pr, "fourier": fourier, "topo": topo,
        "eigenvalues": eigs[:10].tolist()
    }
    
    del L, W, vecs; gc.collect()
    return result, v1, theta, q1, q2

print("✓ Pipeline ready")

In [ ]:
# =============================================================================
# RUN: RATIO SWEEP AT N=20k
# =============================================================================

print("\n" + "#"*60)
print("#  RATIO SWEEP (N=20,000)")
print("#"*60)

N = 20000
ratios = [1.0, 1.18, 1.3, 1.4, 1.6]

results = []
for ratio in ratios:
    r, _, _, _, _ = analyze_fast(N, ratio)
    results.append(r)
    gc.collect()

In [ ]:
# =============================================================================
# RUN: CONVERGENCE TEST
# =============================================================================

print("\n" + "#"*60)
print("#  CONVERGENCE (ratio=1.18)")
print("#"*60)

conv_results = []
for N_test in [5000, 10000, 15000, 20000]:
    r, _, _, _, _ = analyze_fast(N_test, ratio=1.18)
    conv_results.append(r)
    gc.collect()

In [ ]:
# =============================================================================
# SUMMARY TABLES
# =============================================================================

print("\n" + "="*70)
print("  RATIO SWEEP SUMMARY (N=20k)")
print("="*70)
print(f"{'Ratio':>6} | {'λ̂₁×H*':>8} | {'PR':>6} | {'F1':>6} | {'F2':>6} | {'β₁':>8}")
print("-"*55)
for r in results:
    print(f"{r['ratio']:6.2f} | {r['product']:8.4f} | {r['PR']:6.3f} | "
          f"{r['fourier']['F1']:6.3f} | {r['fourier']['F2']:6.3f} | {r['topo']['beta_1']:8d}")

print("\n" + "="*70)
print("  CONVERGENCE SUMMARY (ratio=1.18)")
print("="*70)
print(f"{'N':>7} | {'λ̂₁×H*':>10} | {'σ':>8} | {'PR':>6}")
print("-"*40)
for r in conv_results:
    print(f"{r['N']:7d} | {r['product']:10.4f} | {r['sigma']:8.4f} | {r['PR']:6.3f}")

# Extrapolation
Ns = np.array([r['N'] for r in conv_results])
prods = np.array([r['product'] for r in conv_results])
coeffs = np.polyfit(1/np.sqrt(Ns), prods, 1)
print(f"\n  Fit: λ̂₁×H* = {coeffs[1]:.3f} + {coeffs[0]:.1f}/√N")
print(f"  Limit (N→∞): {coeffs[1]:.3f}")

In [ ]:
# =============================================================================
# VISUALIZATION
# =============================================================================

fig, axes = plt.subplots(2, 3, figsize=(14, 9))

ratios_p = [r['ratio'] for r in results]
products = [r['product'] for r in results]
prs = [r['PR'] for r in results]
f1s = [r['fourier']['F1'] for r in results]
f2s = [r['fourier']['F2'] for r in results]
beta1s = [r['topo']['beta_1'] for r in results]

# 1. Product
ax = axes[0,0]
ax.plot(ratios_p, products, 'o-', ms=10, lw=2)
ax.axhline(13, c='r', ls='--', alpha=0.7, label='13')
ax.set_xlabel('Ratio'); ax.set_ylabel('λ̂₁ × H*')
ax.set_title('Spectral Product', fontweight='bold')
ax.legend(); ax.grid(alpha=0.3)

# 2. PR
ax = axes[0,1]
ax.plot(ratios_p, prs, 's-', ms=10, lw=2, c='purple')
ax.set_xlabel('Ratio'); ax.set_ylabel('PR')
ax.set_title('Participation Ratio', fontweight='bold')
ax.grid(alpha=0.3)

# 3. Fourier
ax = axes[0,2]
ax.plot(ratios_p, f1s, 'o-', ms=8, lw=2, label='F(cos θ)')
ax.plot(ratios_p, f2s, 's-', ms=8, lw=2, label='F(cos 2θ)')
ax.set_xlabel('Ratio'); ax.set_ylabel('Fourier amplitude')
ax.set_title('S¹ Mode Structure', fontweight='bold')
ax.legend(); ax.grid(alpha=0.3)

# 4. β₁
ax = axes[1,0]
ax.plot(ratios_p, beta1s, 'D-', ms=10, lw=2, c='green')
ax.set_xlabel('Ratio'); ax.set_ylabel('β₁')
ax.set_title('Graph Cycles', fontweight='bold')
ax.grid(alpha=0.3)

# 5. Convergence
ax = axes[1,1]
Ns_c = [r['N'] for r in conv_results]
prods_c = [r['product'] for r in conv_results]
ax.plot(Ns_c, prods_c, 'o-', ms=10, lw=2)
ax.axhline(13, c='r', ls='--', alpha=0.7)
ax.axhline(coeffs[1], c='g', ls=':', label=f'Limit: {coeffs[1]:.2f}')
ax.set_xlabel('N'); ax.set_ylabel('λ̂₁ × H*')
ax.set_title('Convergence (r=1.18)', fontweight='bold')
ax.legend(); ax.grid(alpha=0.3)

# 6. Product vs F2
ax = axes[1,2]
sc = ax.scatter(f2s, products, c=ratios_p, s=150, cmap='viridis', edgecolors='k')
plt.colorbar(sc, ax=ax, label='Ratio')
ax.set_xlabel('F(cos 2θ)'); ax.set_ylabel('λ̂₁ × H*')
ax.set_title('Product vs Mode Structure', fontweight='bold')
ax.grid(alpha=0.3)

plt.tight_layout()
plt.savefig('high_n_lite_results.png', dpi=150)
plt.show()
print("✓ Saved: high_n_lite_results.png")

In [ ]:
# =============================================================================
# MODE VISUALIZATION (ratio=1.18)
# =============================================================================

print("\nGenerating mode visualization at ratio=1.18...")
_, v1, theta, q1, q2 = analyze_fast(10000, 1.18, seed=42)

v1_abs = np.abs(v1) / np.max(np.abs(v1))

fig, axes = plt.subplots(1, 3, figsize=(14, 4))

ax = axes[0]
ax.scatter(theta, v1_abs, c=v1_abs, cmap='viridis', s=2, alpha=0.6)
ax.set_xlabel('θ (S¹)'); ax.set_ylabel('|v₁|')
ax.set_title('Mode on S¹', fontweight='bold')

ax = axes[1]
ax.scatter(q1[:,0], v1_abs, c=v1_abs, cmap='viridis', s=2, alpha=0.6)
ax.set_xlabel('q₁[0] (S³₁)'); ax.set_ylabel('|v₁|')
ax.set_title('Mode on S³₁', fontweight='bold')

ax = axes[2]
ax.scatter(q2[:,0], v1_abs, c=v1_abs, cmap='viridis', s=2, alpha=0.6)
ax.set_xlabel('q₂[0] (S³₂)'); ax.set_ylabel('|v₁|')
ax.set_title('Mode on S³₂', fontweight='bold')

plt.tight_layout()
plt.savefig('mode_viz_ratio118.png', dpi=150)
plt.show()
print("✓ Saved: mode_viz_ratio118.png")

In [ ]:
# =============================================================================
# EXPORT
# =============================================================================

export = {
    "timestamp": datetime.now().isoformat(),
    "ratio_sweep": results,
    "convergence": conv_results,
    "extrapolated_limit": float(coeffs[1])
}

with open('high_n_lite_results.json', 'w') as f:
    json.dump(export, f, indent=2)

print("\n" + "="*50)
print("  DONE!")
print("="*50)
print("\nFiles:")
print("  - high_n_lite_results.json")
print("  - high_n_lite_results.png")
print("  - mode_viz_ratio118.png")